In [255]:
import random
import math
import scipy.stats

In [256]:
class Cache:
    
    def __init__(self, cache_size):
        self.cache = []
        self.cache_size = cache_size
    
    def get_element(self, index, remove):
        if (index in self.cache):
            return True
        else:
            self.cache.append(index)
            if (len(self.cache) > cache_size):
                remove(self.cache)
            return False

In [257]:
class Measure:
    
    def __init__(self):
        self.measure_sum = 0.0
        self.measure_sum_sq = 0.0
        self.n = 0
        
    def add_observation(self, obs):
        self.measure_sum += obs
        self.measure_sum_sq += math.pow(obs, 2)
        self.n += 1
        
    def get_num_observations(self):
        return self.n
    
    def sample_mean(self):
        return self.measure_sum / self.n
    
    def sample_variance(self):
        return (self.measure_sum_sq - (math.pow(self.measure_sum, 2) / self.n)) / (self.n - 1)
    
    @staticmethod
    def t(alpha, gl):
        return scipy.stats.t.ppf(1 - (alpha / 2), gl)
    
    def ci_half_width(self, alpha):
        s = math.sqrt(self.sample_variance());
        z = Measure.t(alpha, self.n - 1)
        return z * s / math.sqrt(self.n)

In [258]:
class Exp:
    
  def __init__(self, r):
    self.rate = r

  def next(self):
    return -math.log(random.uniform(0, 1)) / rate

  def exp(self, lam):
    return -math.log(random.uniform(0, 1)) / lam

In [259]:
# Function pointer for fifo eviction policy
def fifo_remove(cache):
    cache.pop(0)

# Function pointer for random eviction policy
def rand_remove(cache):
    random_index = random.randrange(len(cache))
    cache.pop(random_index)

class CacheModellingCT:

    # The warm-up is executed when the cache is set up.
    def __init__(self, warm_up, population, cache_size, cdf):
      self.cache = Cache(cache_size)
      self.population = population
      self.cdf = cdf
      for i in range(warm_up):
        self.make_request()

    def get_item_index(self, p):
      for i in range(self.population):
        if (self.cdf[i] >= p):
            return i + 1
      return self.population

    def make_request(self):
      p = random.uniform(0, 1)
      index = self.get_item_index(p)
      return self.cache.get_element(index, fifo_remove)

    # Reset the measures, but not the state - this is useful when
    # dividing a single run into batches.
    # runLength is the run length of each batch.
    def run(self):
      return self.make_request()

In [260]:
def construct_cdf(population):
    cdf = []
    cumulative = 0.0
    sum_of_lambdas = 0.0
    for k in range(1, population + 1):
        sum_of_lambdas += (1.0 / k)

    for k in range(1, population + 1):
        prob = (1.0 / k) / sum_of_lambdas
        cdf.append(prob + cumulative)
        cumulative += prob

    return cdf

In [261]:
class RunCacheModellingCT:

    def __init__(self, num_batches, num_observations, warm_up, population, cache_size):
        self.num_batches = num_batches
        self.num_observations = num_observations
        self.population = population
        self.warm_up = warm_up
        self.cache_size = cache_size
        self.hit_measure = Measure()
        self.cdf = construct_cdf(population)

    def add_to_measures(self, hit_ratio):
        self.hit_measure.add_observation(hit_ratio)

    def display_results(self):
        hit_ratio = self.hit_measure.sample_mean()
        if (self.num_batches == 1):
            print("Hit ratio: " + ('%.4f' % hit_ratio))
        else:
            hw = self.hit_measure.ci_half_width(0.05)
            lower_ci = hit_ratio - hw
            higher_ci = hit_ratio + hw
            print("Hit ratio:", ('%.4f' % hit_ratio)," CI: (", ('%.4f' % lower_ci), ",", ('%.4f' % higher_ci), ")")
        
    # Each run sets up a fresh board.
    # There is a separate warm-up for each.
    def run_n_times(self):
        for i in range(self.num_batches):
            cache_modelling = CacheModellingCT(self.warm_up, self.population, self.cache_size, self.cdf)
            cur_hit_measure = Measure()
            for j in range(self.num_observations):
                is_hit = 1.0 if cache_modelling.run() else 0.0
                cur_hit_measure.add_observation(is_hit)
            
            cur_hit_ratio = cur_hit_measure.sample_mean()
            self.add_to_measures(cur_hit_ratio)
        
    # Each run resets the measures but doesn't reset the state.
    # There is a single warm-up when the board is built.
    def run_once(self):
        cache_modelling = CacheModellingCT(self.warm_up, self.population, self.cache_size, self.cdf)
        for i in range(self.num_batches):
            cur_hit_measure = Measure()
            for j in range(self.num_observations):
                is_hit = 1.0 if cache_modelling.run() else 0.0
                cur_hit_measure.add_observation(is_hit)
            
            cur_hit_ratio = cur_hit_measure.sample_mean()
            self.add_to_measures(cur_hit_ratio)

In [262]:
def is_within_epsilon(prev_hit_ratios, cur_hit_ratio, epsilon):
    for prev_hit_ratio in prev_hit_ratios:
        if (cur_hit_ratio - epsilon > prev_hit_ratio or prev_hit_ratio > cur_hit_ratio + epsilon):
            return False
    return True

In [263]:
def calculate_warm_up(population, cache_size):
    epsilon = 0.001
    max_warm_up = 100000
    window_size = 10
    ith_run = 1
    prev_hit_ratios = []
    prev_hit_ratios_size = 3
    cache_modelling = CacheModellingCT(0, population, cache_size, construct_cdf(population))
    while (max_warm_up > ith_run):
        cur_hit_measure = Measure()
        for j in range(window_size):
            is_hit = 1.0 if cache_modelling.run() else 0.0
            cur_hit_measure.add_observation(is_hit)

        cur_hit_ratio = cur_hit_measure.sample_mean()
        prev_hit_ratios.append(cur_hit_ratio)
        
        if (len(prev_hit_ratios) == prev_hit_ratios_size):
            if (is_within_epsilon(prev_hit_ratios, cur_hit_ratio, epsilon)):
                break
            else:
                prev_hit_ratios.pop(0)
        ith_run += 1

    return ith_run
    

In [264]:
num_batches = 1000
num_observations_for_one_batch = 1000
population = 1000
cache_size = 10
warm_up = calculate_warm_up(population, cache_size)
sim = RunCacheModellingCT(num_batches, num_observations_for_one_batch, warm_up, population, cache_size)

# sim.run_once()
sim.run_n_times()
sim.display_results()

Hit ratio: 0.1824  CI: ( 0.1816 , 0.1832 )
